In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import inception_v3
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, Reshape
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

2.0.0


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
#converting from csv files to numpy arrays
from numpy import genfromtxt
train_data = genfromtxt("/content/drive/My Drive/cs4990-fall2019-assignment-2/train_data.csv", delimiter = ',')
test_data = genfromtxt("/content/drive/My Drive/cs4990-fall2019-assignment-2/test_data.csv", delimiter = ',')
train_target = genfromtxt("/content/drive/My Drive/cs4990-fall2019-assignment-2/train_target.csv", delimiter = ',')

In [0]:
#printing all loaded data shapes
print(train_data.shape)
print(test_data.shape)
print(train_target.shape)

(16175, 2304)
(3965, 2304)
(16175,)


In [0]:
#early version
model2 = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(3, activation='softmax')
])

model2.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [0]:
#early data preprocessing and shaping
train_data = np.reshape(train_data, (16175, 48, 48, 1))/255
test_data = np.reshape(test_data, (3965, 48, 48, 1))/255
print(test_data.shape)
print(train_data.shape)


(3965, 48, 48, 1)
(16175, 48, 48, 1)


In [0]:
#data preprocessing and shaping for pretrained model
train_data = np.reshape(train_data, (16175, 48, 48))/255
test_data = np.reshape(test_data, (3965, 48, 48))/255
print(test_data.shape)
print(train_data.shape)


(3965, 48, 48)
(16175, 48, 48)


In [0]:
#data preprocessing for pretrained model continued
train_batch = np.repeat(train_data[...,np.newaxis], 3, -1)
print(train_batch.shape)
test_batch = np.repeat(test_data[...,np.newaxis], 3, -1)
print(test_batch.shape)

(16175, 48, 48, 3)
(3965, 48, 48, 3)


In [0]:
#cnn model
model = Sequential()
model.add(Conv2D(32,(3,3), activation='relu', input_shape=(48,48,1)))
model.add(MaxPooling2D(pool_size=(1,1)))

model.add(Conv2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

model.add(Flatten())
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(.5))
model.add(Dense(3, activation='softmax'))

model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 46, 46, 32)        320       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 46, 46, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 44, 44, 32)        9248      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 22, 22, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 20, 20, 64)        18496     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             

In [0]:
#note change epochs to be 35~ if first time in order to get ~.75 validation accuracy
train_datagen = ImageDataGenerator(
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=.2)

test_datagen = ImageDataGenerator()
train_generator = train_datagen.flow(train_data,train_target,batch_size = 32, subset='training')
validation_generator = train_datagen.flow(train_data,train_target,batch_size=32,subset='validation')
model.fit_generator(train_generator,steps_per_epoch=len(train_data)/32,validation_data=validation_generator, epochs=35)

Epoch 1/35
506/505 [==============================] - 40s 78ms/step - loss: 1.0514 - accuracy: 0.4699 - val_loss: 0.9449 - val_accuracy: 0.5586
Epoch 2/35
506/505 [==============================] - 39s 78ms/step - loss: 0.9642 - accuracy: 0.5165 - val_loss: 0.8931 - val_accuracy: 0.5824
Epoch 3/35
506/505 [==============================] - 39s 78ms/step - loss: 0.9262 - accuracy: 0.5534 - val_loss: 0.8615 - val_accuracy: 0.5879
Epoch 4/35
506/505 [==============================] - 39s 78ms/step - loss: 0.9065 - accuracy: 0.5688 - val_loss: 0.8432 - val_accuracy: 0.6025
Epoch 5/35
506/505 [==============================] - 40s 79ms/step - loss: 0.8818 - accuracy: 0.5820 - val_loss: 0.8424 - val_accuracy: 0.5997
Epoch 6/35
506/505 [==============================] - 39s 77ms/step - loss: 0.8614 - accuracy: 0.5981 - val_loss: 0.7926 - val_accuracy: 0.6535
Epoch 7/35
506/505 [==============================] - 39s 78ms/step - loss: 0.8492 - accuracy: 0.6038 - val_loss: 0.8084 - val_accuracy:

In [0]:
#vg model (attemp) does not work
train_datagen=ImageDataGenerator(
        rotation_range=20,
        width_shift_range=.2,
        height_shift_range=.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=.2)
m = inception_v3.InceptionV3(include_top=False, weights='imagenet')
m = Sequential()
m.add(Flatten())
m.add(Dense(256,activation='relu'))
m.add(Dropout(.5))
m.add(Dense(3,activation='softmax'))

m.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])
m.fit(train_batch,train_target,epochs=3,validation_data=)
#generator = train_datagen.flow(train_batch,shuffle=False,subset='training')
#bottleneck_train = m.predict_generator(generator,32)
#np.save(open('bottleneck_features_train.npy', 'w'), bottleneck_train)

#generator = train_datagen.flow(train_batch,shuffle=False,subset='validation')
#bottleneck_validation = m.predict_generator(generator,32)
#np.save(open('bottleneck_features_validation.npy', 'w'), bottleneck_validation)


In [0]:
#fitting old model epochs at 12 gets ~.73
model.fit(train_data, train_target, validation_split = .15, epochs = 35)

Train on 13748 samples, validate on 2427 samples
Epoch 1/35
13748/13748 [==============================] - 7s 526us/sample - loss: 1.0174 - accuracy: 0.4885 - val_loss: 0.8685 - val_accuracy: 0.6148
Epoch 2/35
13748/13748 [==============================] - 6s 430us/sample - loss: 0.8596 - accuracy: 0.5965 - val_loss: 0.7695 - val_accuracy: 0.6663
Epoch 3/35
13748/13748 [==============================] - 6s 422us/sample - loss: 0.7798 - accuracy: 0.6434 - val_loss: 0.7267 - val_accuracy: 0.6786
Epoch 4/35
13748/13748 [==============================] - 6s 427us/sample - loss: 0.7345 - accuracy: 0.6680 - val_loss: 0.6837 - val_accuracy: 0.6951
Epoch 5/35
13748/13748 [==============================] - 6s 425us/sample - loss: 0.6854 - accuracy: 0.6935 - val_loss: 0.6732 - val_accuracy: 0.7025
Epoch 6/35
13748/13748 [==============================] - 6s 422us/sample - loss: 0.6472 - accuracy: 0.7157 - val_loss: 0.6519 - val_accuracy: 0.7244
Epoch 7/35
13748/13748 [===========================

KeyboardInterrupt: ignored

In [0]:
#making prediction of test data and using argmax to make final predictions
predictions = model.predict(test_data)
fp = np.array([])
for row in predictions:
  fp = np.append(fp,np.argmax(row))

In [0]:
#creating id column and merging with prediction
id1 = np.arange(len(predictions))
print(id1.shape)
print(fp.shape)
merged = np.vstack((id1,fp)).T
merged.shape

(3965,)
(3965,)


(3965, 2)

In [0]:
#converting from numpy array to csv
np.savetxt('firstsub.csv', merged, delimiter=',', fmt = '%d')

In [0]:
#changing column names of original csv
#both csv files are saved to root of mounted drive
import pandas as pd
df = pd.read_csv('firstsub.csv', header=None)
df.rename(columns={0:'ID', 1:'Category'}, inplace=True)
df.to_csv('subwname.csv', index=False)